In [3]:
%pip install git+https://github.com/yfukai/BaSiCPy@autosegment

  Cloning https://github.com/yfukai/BaSiCPy (to revision autosegment) to /tmp/pip-req-build-6x9241hx
  Running command git clone --filter=blob:none --quiet https://github.com/yfukai/BaSiCPy /tmp/pip-req-build-6x9241hx
  Running command git checkout -b autosegment --track origin/autosegment
  Switched to a new branch 'autosegment'
  Branch 'autosegment' set up to track remote branch 'autosegment' from 'origin'.
  Resolved https://github.com/yfukai/BaSiCPy to commit 2ebf6939a94919fca8271185e38d0600fb7c1140
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Note: you may need to restart the kernel to use updated packages.


# Install and import 

In [4]:
workdir = "/work/fukai/basicpy"
#os.makedirs(workdir,exist_ok = True)

In [5]:
import numpy as np
import pandas as pd
from basicpy import BaSiC, datasets, metrics
from itertools import product
from tqdm import tqdm
from matplotlib import pyplot as plt
import time
from itertools import product
import pickle 
from os import path
from skimage import filters
import jax
import itertools

plt.rcParams["font.family"] = ""

In [6]:
with open(path.join(workdir,"wo_darkfield_imagess.pickle"),"rb") as f:
    imagess=pickle.load(f)

In [7]:
parameters_df=pd.read_csv(path.join(workdir,"wo_darkfield_parameters.csv"),index_col=0)

In [8]:
_df = parameters_df[parameters_df["true_flatfield_id"]==0]
_imagess = [imagess[i] for i in _df.index.to_numpy()]

In [9]:
estimated_parameters = []
b = BaSiC(
    get_darkfield=False,
)

In [ ]:
jax.config.update('jax_platform_name', 'gpu')

for p,images in tqdm(zip(_df.to_dict(orient="records"),_imagess),total=len(_imagess)):
    for fitting_mode, smoothness_flatfield in itertools.product(
        ["ladmap"],  # fitting_mode
        list(np.logspace(-3, 1, 3)),  # smoothness_flatfield
    ):
        #print(fitting_mode, smoothness_flatfield)
        p2 = p.copy()
        p2.update({
            "fitting_mode": fitting_mode,
            "smoothness_flatfield": smoothness_flatfield,
            "accerelator": "gpu",
        })
        b.fitting_mode=fitting_mode
        b.smoothness_flatfield=smoothness_flatfield
        start = time.time()
        b.fit(np.array(images))
        stop = time.time()
        p2["time"] = stop - start
        estimated_parameters.append(p2)


  0%|                                                                                                                                                                                             | 0/300 [00:00<?, ?it/s]/tmp/ipykernel_140437/2367380063.py:18: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  b.fit(np.array(images))
/tmp/ipykernel_140437/2367380063.py:18: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  b.fit(np.array(images))
  3%|█████▉                                                                                                                                                                            | 10/300 [07:14<2:06:59, 26.27s/it]/tmp/ipykernel_140437/2367380063.py:18: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  b.fit(np.array(

In [1]:
jax.config.update('jax_platform_name', 'cpu')

for p,images in tqdm(zip(_df.to_dict(orient="records"),_imagess),total=len(_imagess)):
    for fitting_mode, smoothness_flatfield in itertools.product(
        ["ladmap"],  # fitting_mode
        list(np.logspace(-3, 1, 3)),  # smoothness_flatfield
    ):
        #print(fitting_mode, smoothness_flatfield)
        p2 = p.copy()
        p2.update({
            "fitting_mode": fitting_mode,
            "smoothness_flatfield": smoothness_flatfield,
            "accerelator": "cpu",
        })
        b.fitting_mode=fitting_mode
        b.smoothness_flatfield=smoothness_flatfield
        start = time.time()
        b.fit(np.array(images))
        stop = time.time()
        p2["time"] = stop - start
        estimated_parameters.append(p2)

NameError: name 'jax' is not defined

In [ ]:
estimated_parameters_df = pd.DataFrame.from_records(estimated_parameters).reset_index(drop=True)
estimated_parameters_df.to_csv("synthesized_data_time_benchmark.csv")